# Step 2: Extracting Comments from r/anime

In [58]:
import time
from time import sleep
import pandas as pd
import datetime as dt
import requests
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from collections import deque, namedtuple
import copy

In [127]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
log = logging.getLogger(__name__)

In [7]:
subreddit = 'anime'
num_posts = 500000
num_days = 360
posts_df = pd.read_csv(f'./data/{subreddit}_submissions_{num_posts}_{num_days}.csv', delimiter=';', header=0)

C:\Users\mattp\anaconda3\envs\nlp_course\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (8,9,56,65,76,82,83,84,85,86,87,88) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Extract Comment IDs from Submission IDs

In [284]:
from pmaw import PushshiftAPI
api = PushshiftAPI()

In [286]:
num_posts = 10000
post_ids = list(posts_df['id'].iloc[:num_posts])
print(f'Extracting comment ids for {num_posts} submissions')

Extracting comment ids for 10000 submissions


In [ ]:
%%time
comment_ids = api.search_submission_comment_ids(ids=post_ids)

In [268]:
num_c_retrieved = len(comment_ids)
print(f'Retrieved {num_c_retrieved}/{num_comments} comments for {num_posts} posts --- {100*num_c_retrieved/num_comments:.2f}%')

Retrieved 150754/162513 comments for 10000 posts --- 92.76%


In [271]:
comments_arr = api.search_comments(ids=comment_ids)

DEBUG:pmaw.PushshiftAPIBase:Created 151 id slices
DEBUG:pmaw.RateLimit:Cache: [1611490790.1957064]- Current Time: 1611490790.1957064
DEBUG:pmaw.RateLimit:Num 1 - 1611490790.1957064 - 0.0
DEBUG:pmaw.PushshiftAPIBase:Time of call: 1611490790.1972048
DEBUG:pmaw.RateLimit:Cache: [1611490790.1957064, 1611490790.2002068]- Current Time: 1611490790.2002068
DEBUG:pmaw.RateLimit:Num 2 - 1611490790.2002068 - 0.004500389099121094
DEBUG:pmaw.RateLimit:Cache: [1611490790.1957064, 1611490790.2002068, 1611490790.2017062]- Current Time: 1611490790.2017062
DEBUG:pmaw.RateLimit:Cache: [1611490790.1957064, 1611490790.2002068, 1611490790.2017062, 1611490790.2027056]- Current Time: 1611490790.2027056
DEBUG:pmaw.RateLimit:Projected 26664.361093452004 -- Desired 60
DEBUG:pmaw.RateLimit:Cache: [1611490790.1957064, 1611490790.2002068, 1611490790.2017062, 1611490790.2027056, 1611490790.2117057]- Current Time: 1611490790.2117057
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.pushshift.io:443


DEBUG:pmaw.RateLimit:Cache: [1611490790.1957064, 1611490790.2002068, 1611490790.2017062, 1611490790.2027056, 1611490790.2117057, 1611490790.214705, 1611490790.2162063, 1611490790.216706, 1611490790.2172046, 1611490790.2177055, 1611490792.7910066]- Current Time: 1611490792.7910066
DEBUG:pmaw.RateLimit:Num 11 - 1611490792.7910066 - 2.5953001976013184
DEBUG:pmaw.RateLimit:Projected 254.30584123177687 -- Desired 60
INFO:pmaw.PushshiftAPIBase:Saving Response
DEBUG:pmaw.PushshiftAPIBase:Imposing rate limit, sleeping for 8.404699802398682
DEBUG:urllib3.connectionpool:https://api.pushshift.io:443 "GET /reddit/comment/search?ids=gj99v8l,gj99voj,gj99wox,gj9a1gc,gj9a3s2,gj9a790,gj9acnw,gj9as51,gj9b0b3,gj9b1a3,gj9b5t3,gj9b7i3,gj9bla7,gj9blon,gj9bqic,gj9c15u,gj9ccam,gj9cs1u,gj9csgt,gj9cwsh,gj9d0oq,gj9d4oe,gj9dexb,gj9dn42,gj9ecoa,gj9ezqn,gj9f2mu,gj9f8se,gj9fbg7,gj9fdbl,gj9fft4,gj9fk12,gj9fkzh,gj9fr0k,gj9fsts,gj9fxmk,gj9g0w2,gj9g17s,gj9g1bo,gj9g8sp,gj9gds1,gj9gek6,gj9gip8,gj9gk4z,gj9grfa,gj9guwj,gj9g

DEBUG:pmaw.RateLimit:Cache: [1611490790.1957064, 1611490790.2002068, 1611490790.2017062, 1611490790.2027056, 1611490790.2117057, 1611490790.214705, 1611490790.2162063, 1611490790.216706, 1611490790.2172046, 1611490790.2177055, 1611490792.7910066, 1611490795.0248659]- Current Time: 1611490795.0248659
DEBUG:pmaw.RateLimit:Num 12 - 1611490795.0248659 - 4.829159498214722
DEBUG:pmaw.RateLimit:Projected 149.09426790856148 -- Desired 60
INFO:pmaw.PushshiftAPIBase:Saving Response
DEBUG:pmaw.PushshiftAPIBase:Imposing rate limit, sleeping for 7.170840501785278
DEBUG:pmaw.PushshiftAPIBase:Time of call: 1611490796.2197232
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.pushshift.io:443
DEBUG:pmaw.PushshiftAPIBase:Time of call: 1611490797.2190487
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.pushshift.io:443
DEBUG:urllib3.connectionpool:https://api.pushshift.io:443 "GET /reddit/comment/search?ids=gj8y55g,gj8ya45,gj8ybq2,gj8yhsy,gj8ynz2,gj8yz2p,gj8z8la,gj8zp

DEBUG:pmaw.RateLimit:Cache: [1611490790.1957064, 1611490790.2002068, 1611490790.2017062, 1611490790.2027056, 1611490790.2117057, 1611490790.214705, 1611490790.2162063, 1611490790.216706, 1611490790.2172046, 1611490790.2177055, 1611490792.7910066, 1611490795.0248659, 1611490798.5331297]- Current Time: 1611490798.5331297
INFO:pmaw.PushshiftAPIBase:Saving Response
DEBUG:pmaw.RateLimit:Num 13 - 1611490798.5331297 - 8.337423324584961
DEBUG:pmaw.RateLimit:Projected 93.5540837539071 -- Desired 60
DEBUG:pmaw.PushshiftAPIBase:Imposing rate limit, sleeping for 4.662576675415039
DEBUG:urllib3.connectionpool:https://api.pushshift.io:443 "GET /reddit/comment/search?ids=gja0vci,gjabn5s,gj8clvx,gj8cbk0,gj8c0p6,gj8c1kb,gj8cotx,gj8cp1z,gj8s4jh,gj8t11o,gj8bqif,gj8dqtp,gj8e19z,gj8gy33,gj8gze1,gj8b29i,gj8bq8t,gj8btx7,gj8bv0s,gj8bzys,gj8c7qu,gj8csbc,gj8djh7,gj8dtck,gj8e7ry,gj8fosq,gj8gr5u,gj8hu0u,gj8izjk,gj8m75r,gj8mlu0,gj8nxx5,gj8p6h2,gj8pho5,gj8rgbn,gj8sgya,gj8tagm,gj8wirg,gj9fkn7,gjlnrwe,gj83k34,gj895rn

DEBUG:pmaw.RateLimit:Cache: [1611490790.1957064, 1611490790.2002068, 1611490790.2017062, 1611490790.2027056, 1611490790.2117057, 1611490790.214705, 1611490790.2162063, 1611490790.216706, 1611490790.2172046, 1611490790.2177055, 1611490792.7910066, 1611490795.0248659, 1611490798.5331297, 1611490799.7267177]- Current Time: 1611490799.7267177
INFO:pmaw.PushshiftAPIBase:Saving Response
DEBUG:pmaw.RateLimit:Num 14 - 1611490799.7267177 - 9.53101134300232
DEBUG:pmaw.RateLimit:Projected 88.13335434929779 -- Desired 60
DEBUG:pmaw.PushshiftAPIBase:Imposing rate limit, sleeping for 4.468988656997681
DEBUG:pmaw.PushshiftAPIBase:Time of call: 1611490800.2196705
DEBUG:pmaw.PushshiftAPIBase:Time of call: 1611490800.225165
DEBUG:pmaw.PushshiftAPIBase:Time of call: 1611490800.225666
DEBUG:pmaw.PushshiftAPIBase:Time of call: 1611490800.225666
DEBUG:pmaw.PushshiftAPIBase:Time of call: 1611490800.225666
DEBUG:pmaw.PushshiftAPIBase:Time of call: 1611490800.2261405
DEBUG:urllib3.connectionpool:Starting new H

...

DEBUG:pmaw.RateLimit:Removing Time: 1611490883.4342802 -- 1611490945.6193936 -- -62.18511343002319
DEBUG:pmaw.RateLimit:Removing Time: 1611490883.4342802 -- 1611490945.6198938 -- -62.18561363220215
DEBUG:pmaw.RateLimit:Removing Time: 1611490883.4342802 -- 1611490945.6203938 -- -62.186113595962524
DEBUG:pmaw.RateLimit:Removing Time: 1611490883.4342802 -- 1611490945.620894 -- -62.18661379814148
INFO:pmaw.RateLimit:1611490881.921875 has already been removed
INFO:pmaw.RateLimit:1611490881.921875 has already been removed
DEBUG:pmaw.RateLimit:Removing Time: 1611490883.958489 -- 1611490945.6193936 -- -61.6609046459198
INFO:pmaw.RateLimit:1611490883.4342802 has already been removed
INFO:pmaw.RateLimit:1611490883.4342802 has already been removed
INFO:pmaw.RateLimit:1611490883.4342802 has already been removed
DEBUG:pmaw.RateLimit:Removing Time: 1611490883.958489 -- 1611490945.6193936 -- -61.6609046459198
DEBUG:pmaw.RateLimit:Removing Time: 1611490883.958489 -- 1611490945.6198938 -- -61.661404848

DEBUG:urllib3.connectionpool:https://api.pushshift.io:443 "GET /reddit/comment/search?ids=gig8hx0,gig8mi2,gig8tzo,gig904i,gig9bi7,gig9ch9,gig9f4f,gig9mhc,gig9oos,gig9pu4,gig9rl6,gig9rlr,gig9vis,giga0t7,gigadwk,gigaliy,gigarnz,gigawna,gigax2l,gigb54y,gigbk8s,gigc9cu,gigcnp0,gigcpj4,gigcq9g,gigcr3m,gigcr43,gigd8hs,gigd8sk,gigdtei,gige1u4,gige7sl,gigejy0,gigf8qv,giggi39,giggpyt,gigh73b,gigh903,gighbsl,gighccn,gighoxu,gigi0iu,gigimcl,gigisn9,gigj665,gigjkjk,gigl1qw,giglkfk,gigln3j,giglotj,giglpx8,giglzwz,gigm3zb,gigm4d2,gigm7b6,gigm871,gigmas1,gigmw63,gignepv,gigodsr,gigop4y,gigoyuv,gigp3t6,gigppbv,gigpsy6,gigqdmx,gigqvdg,gigr0bg,gigr1v0,gigr9i8,gigrwyx,gigrzbm,gigs3fv,gigs8ya,gigt02i,gigt8qn,gigtdjy,gigti7s,gigtmk3,gigtrig,gigu453,gigu7w0,gigueqv,giguk7z,giguo5o,gigvprc,gigvsy5,gigwpfb,gigygff,gigykyf,gigz6ld,gigzhic,gigzlyh,gigzzps,gih04ep,gih0br6,gih0eg1,gih0ymh,gih1cst,gih1wuu,gih27x8,gih2gy1,gih2m06,gih2ygk,gih35y9,gih3oiw,gih40n1,gih463d,gih4amw,gih56pl,gih5n5n,gih5qjj,gih5sr8,gih5xi

DEBUG:urllib3.connectionpool:https://api.pushshift.io:443 "GET /reddit/comment/search?ids=gixt59q,gixt6ip,gixt7ew,gixt8u0,gixtb60,gixtcik,gixtfih,gixtfnv,gixtg25,gixtgw2,gixtgyk,gixtk5a,gixtm96,gixtoh1,gixtowd,gixtowh,gixtswa,gixttxx,gixtxrw,gixu3sv,gixu4dd,gixu5nd,gixu5ru,gixu5w8,gixu6jq,gixua3e,gixuan8,gixuawl,gixuc06,gixuc9w,gixuiwz,gixukbj,gixuld6,gixuqpv,gixur4h,gixurs0,gixurtr,gixuspa,gixustg,gixutpc,gixuuzf,gixuwjd,gixv11p,gixv1uk,gixv3aj,gixv3pb,gixv4nq,gixv6fb,gixva4f,gixvb0o,gixvg47,gixvh0p,gixvhwh,gixvick,gixvilt,gixvjw9,gixvko4,gixvmr9,gixvmuv,gixvodg,gixvuag,gixvuk5,gixw48x,gixw4nl,gixw4tb,gixw6c5,gixwbb3,gixwbyn,gixwe18,gixwjex,gixwm8e,gixwpzn,gixwqth,gixwssx,gixx6tl,gixxb3d,gixxb4h,gixxbjv,gixxcmi,gixxcn6,gixxee1,gixxfa1,gixxff0,gixxh6m,gixxjem,gixxkyy,gixxmmf,gixxonv,gixxriq,gixxs0q,gixxsg6,gixxuyx,gixxx4k,gixxxlj,gixxxmo,gixxyf9,gixxztx,gixy02p,gixy15o,gixy3if,gixy4oj,gixy6uw,gixyavb,gixyaz2,gixycol,gixyd6p,gixye2m,gixygea,gixygti,gixyh2i,gixyh3z,gixyhcq,gixyi2a,gixyix

DEBUG:urllib3.connectionpool:https://api.pushshift.io:443 "GET /reddit/comment/search?ids=ghds1gc,ghdsu4h,ghdu7ns,ghduubs,ghdv3p8,ghdv4q1,ghdv9t4,ghdvi79,ghdvi7z,ghdvns4,ghdw2jp,ghdwcyf,ghdxnzb,ghdxz2h,ghdyslg,ghdyvtl,ghdzjuc,ghe05n8,ghe090k,ghe0d6w,ghe0o9x,ghe21f1,ghe2iei,ghe5ukc,ghe8d5z,ghe8q7b,gheawgc,gheb076,gheeedc,gheguam,gheh4pi,ghelad4,ghelxtn,ghen1cs,ghep2nn,ghep5f4,ghepzc0,gheqnrm,gheqqhc,gher33d,ghev168,ghf14k5,ghf2uqf,ghf5kso,ghfe0uz,ghfmh5e,ghfowmt,ghfvzj4,ghfw736,ghfxdew,ghfyou0,ghg1v2d,ghg4lso,ghg94qo,ghgjh4z,ghgkjay,ghgz6ta,ghh016k,ghh0vs1,ghh15eu,ghh6jqm,ghh71du,ghi1pds,ghii8mb,ghiz6lm,gid2rrf,gid2zbn,gifm1o0,gifmey4,ghbu7w2,ghbnduc,ghbnipr,ghbntn9,ghbrlxp,ghbrnee,ghbrntv,ghbroog,ghbrqp1,ghbrs1u,ghbrw64,ghbryig,ghbrza4,ghbrzsg,ghbs23l,ghbs2i8,ghbs8yr,ghbs9io,ghbsd6r,ghbsf1j,ghbsfwu,ghbsgc0,ghbsptu,ghbsvrx,ghbt1he,ghbt2g4,ghbt8m8,ghbta3d,ghbtb1g,ghbtg48,ghbtiln,ghbtj3g,ghbtlu9,ghbtnbi,ghbtt46,ghbttzn,ghbu2xd,ghbuha6,ghbuzah,ghbv3wk,ghbv7jl,ghbv7y9,ghbvjqj,ghbvpa8,ghbw34

INFO:pmaw.PushshiftAPIBase:Saving Response
INFO:pmaw.PushshiftAPIBase:Saving Response
INFO:pmaw.PushshiftAPIBase:Saving Response
INFO:pmaw.PushshiftAPIBase:Saving Response
Total Success Rate: 88.82% -- Total Reqs: 170 -- Num Retries: 2


In [272]:
print(f'Retrieved {len(comments_arr)} comments')

Retrieved 150754 comments


In [273]:
comments_df = pd.DataFrame(comments_arr)

In [274]:
comments_df.head(5)

,all_awardings,approved_at_utc,associated_award,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,...,retrieved_on,score,send_replies,stickied,subreddit,subreddit_id,top_awarded_type,total_awards_received,treatment_tags,author_cakeday
0,[],None,None,AutoModerator,None,None,[],None,None,None,...,1610724150,1,False,True,anime,t5_2qh22,None,0,[],NaN
1,[],None,None,BoopTheChicken,None,None,[],None,None,None,...,1610725098,2,True,False,anime,t5_2qh22,None,0,[],NaN
2,[],None,None,THEbaddestOFtheASSES,None,None,[],None,None,None,...,1610725376,3,True,False,anime,t5_2qh22,None,0,[],NaN
3,[],None,None,[deleted],,None,NaN,None,None,dark,...,1610725780,2,True,False,anime,t5_2qh22,None,0,[],NaN
4,[],None,None,AutoModerator,None,None,[],None,None,None,...,1610725944,1,False,False,anime,t5_2qh22,None,0,[],NaN


### Save Comments

In [277]:
print(f'Saving comments for {num_posts} posts from r/{subreddit}')

for index, row in comments_df.iterrows():
    row['body'] = re.sub(r';+', '.', row['body'])
    
comments_df.to_csv(f'./data/{subreddit}_comments_{num_posts}.csv', sep=';', header=True, index=False, columns=list(comments_df.axes[1]))

Saving comments for 10000 posts from r/anime
